In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.data import AUTOTUNE
import os



IMG_SIZE = (256, 256)
BATCH = 32

train_ds = keras.utils.image_dataset_from_directory(
    "data3a/training", labels="inferred", label_mode="int",
    image_size=IMG_SIZE, shuffle=True
)
val_ds = keras.utils.image_dataset_from_directory(
    "data3a/validation", labels="inferred", label_mode="int",
    image_size=IMG_SIZE, shuffle=False
)

# (optional) use val as test for now
test_ds = val_ds

class_names = train_ds.class_names
num_classes = len(class_names)

# cache/prefetch
train_ds = train_ds.cache().prefetch(AUTOTUNE)
val_ds   = val_ds.cache().prefetch(AUTOTUNE)
test_ds  = test_ds.cache().prefetch(AUTOTUNE)

# normalize to 0..1
normalization = keras.layers.Rescaling(1./255)
train_ds_n = train_ds.map(lambda x,y: (normalization(x), y))
val_ds_n   = val_ds.map(lambda x,y: (normalization(x), y))
test_ds_n  = test_ds.map(lambda x,y: (normalization(x), y))


In [ ]:
input_shape = (256, 256, 3)

base_model = keras.applications.VGG16(
    include_top=False, weights="imagenet", input_shape=input_shape
)
base_model.trainable = False  # start frozen

inputs = keras.Input(shape=input_shape)
x = base_model(inputs, training=False)
x = keras.layers.GlobalAveragePooling2D()(x)

x = keras.layers.Dense(512, activation="relu")(x)
x = keras.layers.BatchNormalization()(x)
x = keras.layers.Dropout(0.4)(x)

x = keras.layers.Dense(256, activation="relu")(x)
x = keras.layers.BatchNormalization()(x)
x = keras.layers.Dropout(0.3)(x)

x = keras.layers.Dense(128, activation="relu")(x)
x = keras.layers.BatchNormalization()(x)

outputs = keras.layers.Dense(num_classes, activation="softmax")(x)
model = keras.Model(inputs, outputs)
model.summary()


In [ ]:
os.makedirs("models", exist_ok=True)

early_stop = keras.callbacks.EarlyStopping(
    monitor="val_accuracy", patience=5, restore_best_weights=True
)
ckpt = keras.callbacks.ModelCheckpoint(
    filepath="models/vgg16_finetuned_best.keras",  # NOTE: .keras extension
    monitor="val_accuracy", save_best_only=True, verbose=1
)
reduce_lr = keras.callbacks.ReduceLROnPlateau(
    monitor="val_loss", factor=0.5, patience=3, verbose=1
)

model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=0.01),
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"]
)

train_input = train_ds_n_aug if "train_ds_n_aug" in globals() else train_ds_n

history = model.fit(
    train_input,
    validation_data=val_ds_n,
    epochs=20,
    callbacks=[early_stop, ckpt, reduce_lr],
    verbose=1
)


In [ ]:
base_model.trainable = True
for layer in base_model.layers[:-4]:
    layer.trainable = False

model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=1e-5),
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"]
)

history_ft = model.fit(
    train_input,
    validation_data=val_ds_n,
    epochs=20,
    callbacks=[early_stop, ckpt, reduce_lr],
    verbose=1
)


In [ ]:
test_loss, test_acc = model.evaluate(test_ds_n, verbose=0)
print(f"Test Acc: {test_acc:.3f}  |  Test Loss: {test_loss:.3f}")

# Confusion matrix
import numpy as np
from sklearn.metrics import confusion_matrix, classification_report

y_true = np.concatenate([y.numpy() for _, y in test_ds_n])
y_pred_probs = model.predict(test_ds_n, verbose=0)
y_pred = np.argmax(y_pred_probs, axis=1)

print(classification_report(y_true, y_pred, target_names=class_names))

cm = confusion_matrix(y_true, y_pred)
print(cm)


In [ ]:
import os
os.makedirs("models", exist_ok=True)

# Save the final full Keras model (recommended)
model.save("models/vgg16_finetuned_final.keras")   # <-- note the .keras extension

# (Optional) also export a TF SavedModel directory for deployment
model.export("models/vgg16_finetuned_savedmodel")  # creates a folder


In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# pick the correct history object
hist = history_ft if 'history_ft' in globals() else history

train_acc = hist.history["accuracy"]
val_acc   = hist.history["val_accuracy"]
train_loss = hist.history["loss"]
val_loss   = hist.history["val_loss"]
epochs = np.arange(1, len(train_acc)+1)

fig, ax = plt.subplots(1, 2, figsize=(12, 4))

ax[0].plot(epochs, train_acc, label="Train")
ax[0].plot(epochs, val_acc,   label="Validation")
ax[0].set_title("Accuracy"); ax[0].set_xlabel("Epoch"); ax[0].legend()

ax[1].plot(epochs, train_loss, label="Train")
ax[1].plot(epochs, val_loss,   label="Validation")
ax[1].set_title("Loss"); ax[1].set_xlabel("Epoch"); ax[1].legend()

plt.tight_layout()
plt.savefig("models/training_curves.png", dpi=150)
plt.show()


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, classification_report

# if you restored best weights via EarlyStopping that's fine.
# Otherwise, you can ensure best checkpoint by uncommenting:
# from tensorflow import keras
# model = keras.models.load_model("models/vgg16_finetuned_best.keras")

# Ensure these exist from your earlier cells:
# class_names, test_ds_n, model

y_true = np.concatenate([y.numpy() for _, y in test_ds_n])
y_prob = model.predict(test_ds_n, verbose=0)
y_pred = np.argmax(y_prob, axis=1)

print(classification_report(y_true, y_pred, target_names=class_names))

cm = confusion_matrix(y_true, y_pred)
disp = ConfusionMatrixDisplay(cm, display_labels=class_names)

fig2, ax2 = plt.subplots(figsize=(5,5))
disp.plot(values_format='d', cmap="Blues", ax=ax2, colorbar=False)
plt.title("Confusion Matrix")
plt.tight_layout()
plt.savefig("models/confusion_matrix.png", dpi=150)
plt.show()
